In [1]:
#!pip install mxnet
#!pip install gluonnlp pandas tqdm
#!pip install sentencepiece
#!pip install transformers==3.0.2
#!pip install torch

In [2]:
#!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
#pip install openpyxl

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/piai/test/Project_NLP/Model_training_yes/.cache/kobert_v1.zip
using cached model. /home/piai/test/Project_NLP/Model_training_yes/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
import pandas as pd
import os
import openpyxl
chatbot_data = pd.read_excel('/home/piai/test/Project_NLP/Model_training_yes/네_데이터셋.xlsx')

In [8]:
chatbot_data.sample(n=10)

,answer,output
40,음.. 아니요,아니
43,아니 안 먹어,아니
30,아니요 안먹을게요,아니
49,아니요 안 먹을래요,아니
8,네~,네
10,음.. 네,네
22,내 먹으려고요,네
17,내 먹을게요,네
47,아녜요,아니
48,아뇨 괜찮아요,아니


In [9]:
chatbot_data.loc[(chatbot_data['output'] == "네"), 'output'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['output'] == "아니"), 'output'] = 1  #놀람 => 1


data_list = []
for q, label in zip(chatbot_data['answer'], chatbot_data['output'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

In [10]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)
#print(len(dataset_train))
#print(len(dataset_test))


In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
# Setting parameters
max_len = 64
batch_size = 10
warmup_ratio = 0.1
num_epochs = 30
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /home/piai/test/Project_NLP/Model_training_yes/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [15]:
class BERTClassifier_yes(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier_yes, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
#BERT 모델 불러오기
model_yes = BERTClassifier_yes(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model_yes.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model_yes.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [17]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model_yes.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_yes(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_yes.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model_yes.eval()
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model_yes(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)
        print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_23420/2475456449.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/4 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.1311719417572021 train acc 0.1
epoch 1 train acc 0.25


/tmp/ipykernel_23420/2475456449.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/2 [00:00<?, ?it/s]

...

epoch 29 test acc 1.0


  0%|          | 0/4 [00:00<?, ?it/s]

epoch 30 batch id 1 loss 0.009270950220525265 train acc 1.0
epoch 30 train acc 1.0


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 30 test acc 1.0


In [18]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict_yes(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model_yes.eval()
    
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)

            valid_length= valid_length
            label = label.long().to(device)

            out = model_yes(token_ids, valid_length, segment_ids)


            test_eval=[]
            for i in out:
                logits=i
                logits = logits.detach().cpu().numpy()

                if np.argmax(logits) == 0:
                    test_eval.append("네 그럼 오늘의 메뉴를 불러 드리겠습니다")
                elif np.argmax(logits) == 1:
                    test_eval.append("알겠습니다. 추가요청사항이 있으면 승무원을 불러주세요")

            print(test_eval[0])

using cached model. /home/piai/test/Project_NLP/Model_training_yes/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [19]:
torch.save(model_yes.state_dict(), 'model_yes.pth')

In [22]:
'''연습
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("식사하시겠습니까? : ")
    if sentence == '0' :
        break
    predict_yes(sentence)
    print("\n")
'''

'연습\n#질문 무한반복하기! 0 입력시 종료\nend = 1\nwhile end == 1 :\n    sentence = input("식사하시겠습니까? : ")\n    if sentence == \'0\' :\n        break\n    predict_yes(sentence)\n    print("\n")\n'